Antes de mais nada os notebooks aqui mostrado tiveram como base/foram retirados dos seguintes repositórios: 
 > https://github.com/fchollet/deep-learning-with-python-notebooks 
 
 
 > https://github.com/cdfmlr/Deep-Learning-with-Python-Notebooks
 
 Sugiro fortemente que consultem os códigos originais e em caso de dúvida podem me contatar para conversarmos. 

# Aprendizado profundo com Python

## 8.3 Transferência de estilo neural

> Transferência de estilo neural

A transferência de estilo neural, uma rede neural baseada no aprendizado profundo, aplica o estilo da imagem de referência à imagem alvo enquanto preserva o conteúdo da imagem alvo para criar uma nova imagem.



A ideia de transferência de estilo neural é simples: defina uma função de perda para especificar o objetivo a ser alcançado e, em seguida, minimize essa perda. O objetivo aqui é preservar o conteúdo da imagem original, ao mesmo tempo que adota o estilo da imagem de referência.

Supondo que existam funções de conteúdo e estilo que podem calcular o conteúdo e o estilo da imagem de entrada, respectivamente, e a distância da função de forma normal, a perda de transferência de estilo neural pode ser expressa como:

`` `python
perda = distância (conteúdo (imagem_original) -content (imagem_gerada)) +
       distance (style (reference_image) -style (generated_image))
`` `

Na verdade, usando redes neurais convolucionais profundas, as funções de estilo e conteúdo podem ser definidas matematicamente.

#### Definição de perda

1. ** Perda de conteúdo **

As ativações de camadas próximas à parte inferior (frente) da rede neural convolucional contêm informações locais sobre a imagem, e as camadas próximas à parte superior (traseira) contêm informações mais globais e abstratas. O conteúdo é a informação global e abstrata da imagem, portanto, a ativação da camada superior da rede neural convolucional pode ser usada para representar o conteúdo da imagem.

Portanto, dada uma rede neural convolucional pré-treinada, selecione uma camada superior, a perda de conteúdo pode usar o L2 entre "a ativação desta camada na imagem alvo" e "a ativação desta camada na imagem gerada" Norma.

2. ** Perda de estilo **

Ao contrário do conteúdo que pode ser expresso com apenas uma camada, o estilo requer várias camadas para definir. Existem muitos estilos de estilos, como pinceladas, linhas, texturas, cores, etc. Esses conteúdos aparecerão em diferentes níveis de abstração. Portanto, a expressão do estilo precisa capturar a aparência extraída em todas as escalas espaciais, não apenas em uma única escala.

Sob esse tipo de pensamento, a expressão da perda de estilo pode contar com a matriz de Gram da ativação da camada. Esta matriz de Gram é o produto interno de cada mapa de feições de uma determinada camada, expressando o mapeamento da correlação entre as feições da camada, e corresponde à aparência da textura encontrada nesta escala. A preservação de relações internas semelhantes em diferentes ativações pode ser considerada como "estilo".

Então, podemos usar as texturas mantidas em diferentes camadas da imagem gerada e a imagem de referência de estilo para definir a perda de estilo.

#### Implementação de Keras de transferência de estilo neural

A transferência de estilo neural pode ser implementada com qualquer rede neural convolucional pré-treinada e o VGG19 é usado aqui.

As etapas de transferência de estilo neural são as seguintes:

1. Crie uma rede para calcular simultaneamente a ativação da camada VGG19 da imagem de referência de estilo, imagem alvo e imagem gerada;
2. Use as ativações de camada calculadas nessas três imagens para definir a função de perda descrita anteriormente;
3. Descida em gradiente para minimizar esta função de perda.

Antes de começar a construir a rede, defina o caminho da imagem de referência do estilo e a imagem alvo. Se o tamanho da imagem for muito diferente, a transferência do estilo será mais difícil, então aqui também definimos o tamanho:

In [15]:
# Não use o modo de execução just-in-time

import tensorflow as tf

tf.compat.v1.disable_eager_execution()

In [16]:
# Defina as variáveis ​​iniciais

from tensorflow.keras.preprocessing.image import load_img, img_to_array

target_image_path = 'img/portrait.jpg'
style_referencce_image_path = 'img/transfer_style_reference.jpg'

width, height = load_img(target_image_path).size
img_height = 400
img_width = width * img_height // height

Eu escolhi esta foto:

-transfer_style_reference: Vincent Van Gogh, "A Wheatfield, with Cypresses" (* A Wheatfield, with Cypresses *), 1889, coletado no Metropolitan Museum of New York.
-poretrato: Paul Gauguin "The Swineherd, Brittany" (* The Swineherd, Brittany *), 1888, coletado no Museu de Arte do Condado de Los Angeles, Califórnia, EUA.


A seguir, precisamos de algumas funções auxiliares para carregamento e processamento de imagens.

In [17]:
# Função auxiliar

import numpy as np
from tensorflow.keras.applications import vgg19

def preprocess_image(image_path):
    img = load_img(image_path, target_size=(img_height, img_width))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    return img

def deprocess_image(x):
    # vgg19.preprocess_input O valor médio do pixel de ImageNet é subtraído para tornar seu centro zero. Faça a operação inversa aqui:
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.680
    
    # BGR -> RGB
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    
    return x

Em seguida, construa a rede VGG19: Receba lote de três imagens como entrada.As três imagens são a imagem de referência de estilo, a constante da imagem de destino e um espaço reservado para salvar a imagem gerada.

In [18]:
from tensorflow.keras import backend as K

target_image = K.constant(preprocess_image(target_image_path))
style_reference_image = K.constant(preprocess_image(style_referencce_image_path))
combination_image = K.placeholder((1, img_height, img_width, 3))

input_tensor = K.concatenate([target_image,
                              style_reference_image,
                              combination_image], axis=0)

model = vgg19.VGG19(input_tensor=input_tensor,
                    weights='imagenet',
                    include_top=False)

print('Model loaded.')

Model loaded.


Defina a perda de conteúdo para garantir que a imagem de destino e a imagem gerada tenham resultados semelhantes no topo da rede:

In [19]:
# Perda de conteúdo

def content_loss(base, combination):
    return K.sum(K.square(combination - base))

Depois, há a perda de estilo, que calcula a matriz de Gram da matriz de entrada e calcula a perda de estilo usando a matriz de Gram:

In [20]:
# Perda de estilo

def gram_matrix(x):
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram

def style_loss(style, combination):
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_height * img_width
    return K.sum(K.square(S - C)) / (4.0 * (channels ** 2) * (size ** 2))

Aqui definimos adicionalmente uma "perda total de variação" para promover a continuidade espacial da imagem gerada e evitar pixelização excessiva do resultado, o que equivale a uma regularização.

In [21]:
# Perda de variação total

def total_variation_loss(x):
    a = K.square(
        x[:, :img_height - 1, :img_width - 1, :] - 
        x[:, 1:, :img_width - 1, :])
    b = K.square(
        x[:, :img_height - 1, :img_width - 1, :] - 
        x[:, :img_height - 1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))

Agora, considere o cálculo de perda específico: ao calcular a perda de conteúdo, precisamos de uma camada superior; para a perda de estilo, precisamos usar uma série de camadas, incluindo as camadas superior e inferior; e, finalmente, precisamos adicionar a perda de variação total. A perda final é a média ponderada desses três tipos de perdas.

In [22]:
# Defina a perda final que precisa ser minimizada

outputs_dict = {layer.name: layer.output for layer in model.layers}

content_layer = 'block5_conv2'
style_layers = [f'block{i}_conv1' for i in range(1, 6)]

total_variation_weight = 1e-4
style_weight = 1.0
content_weight = 0.025  # Quanto maior o content_weight, mais fácil é o conteúdo de destino, mais fácil é identificar na imagem gerada

# Perda de conteúdo
loss = K.variable(0.)
layer_features = outputs_dict[content_layer]
target_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]
loss = loss + content_weight * content_loss(target_image_features, combination_features)

# Perda de estilo
for layer_name in style_layers:
    layer_features = outputs_dict[layer_name]
    style_reference_features = layer_features[1, :, :, :]
    combination_features = layer_features[2, :, :, :]
    sl = style_loss(style_reference_features, combination_features)
    loss = loss + (style_weight / len(style_layers)) * sl
    
# Perda de variação total
loss = loss + total_variation_weight * total_variation_loss(combination_image)

A última etapa é o processo de descida gradiente. Aqui chamamos scipy e usamos o algoritmo `L-BFGS` para otimização.

Para um cálculo rápido, criamos uma classe Evaluator para calcular o valor da perda e o valor do gradiente ao mesmo tempo. O valor da perda será retornado quando a primeira chamada for feita e o valor do gradiente será armazenado em cache para a próxima chamada.

In [23]:
# Configure o processo de descida gradiente

grads = K.gradients(loss, combination_image)[0]
fetch_loss_and_grads = K.function([combination_image], [loss, grads])

class Evaluator(object):
    def __init__(self):
        self.loss_value = None
        self.grads_values = None
        
    def loss(self, x):
        assert self.loss_value is None
        x = x.reshape((1, img_height, img_width, 3))
        outs = fetch_loss_and_grads([x])
        loss_value = outs[0]
        grad_values = outs[1].flatten().astype('float64')
        self.loss_value = loss_value
        self.grads_values = grad_values
        return self.loss_value
    
    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grads_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values
        
evaluator = Evaluator()

Finalmente, chame o algoritmo L-BFGS de SciPy para executar o processo de subida de gradiente e salve a imagem gerada atual após cada iteração (subida de gradiente de 20 etapas):

In [24]:
from scipy.optimize import fmin_l_bfgs_b
from imageio import imsave
import time

iterations = 20

def result_fname(iteration):
    return f'results/result_at_iteration_{iteration}.png'

x = preprocess_image(target_image_path)
x = x.flatten()

for i in range(iterations):
    print('Start of iteration', i)
    start_time = time.time()
    
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss,
                                     x,
                                     fprime=evaluator.grads,
                                     maxfun=20)
    print('  Current loss value:', min_val)
    
    img = x.copy().reshape((img_height, img_width, 3))
    img = deprocess_image(img)
    fname = result_fname(i)
    imsave(fname, img)
    print('  Image saved as', fname)
    
    end_time = time.time()
    print(f'  Iteration {i} completed in {end_time - start_time} s')

Start of iteration 0
  Current loss value: 442468450.0
  Image saved as results/result_at_iteration_0.png
  Iteration 0 completed in 177.57321500778198 s
Start of iteration 1
  Current loss value: 220311460.0
  Image saved as results/result_at_iteration_1.png
  Iteration 1 completed in 170.76263999938965 s
Start of iteration 2
  Current loss value: 161225440.0
  Image saved as results/result_at_iteration_2.png
  Iteration 2 completed in 181.72589802742004 s
Start of iteration 3
  Current loss value: 135288640.0
  Image saved as results/result_at_iteration_3.png
  Iteration 3 completed in 176.2833709716797 s
Start of iteration 4
  Current loss value: 117499910.0
  Image saved as results/result_at_iteration_4.png
  Iteration 4 completed in 191.78947687149048 s
Start of iteration 5
  Current loss value: 105684536.0
  Image saved as results/result_at_iteration_5.png
  Iteration 5 completed in 189.79483294487 s
Start of iteration 6
  Current loss value: 95785740.0
  Image saved as results/r

Compare o resultado com a imagem original:

! [Resultado 1] (https://tva1.sinaimg.cn/large/007S8ZIlgy1ghu38838haj317h0u0b2b.jpg)

Vejamos outro exemplo: "Cypress Tree in a Rye" de Van Gogh é usado para referência de estilo, e Miller "Gleaner" (Des glaneuses, 1857, Museu Orsay de Paris) é usado para o conteúdo. O que é mais interessante é que o próprio Van Gogh fez um desenho de Zwei grabende Bäuerinnen auf schneebedecktem Feld (Zwei grabende Bäuerinnen auf schneebedecktem Feld) que imitava parcialmente o "Grão Gleaner":

! [Resultado 2] (https://tva1.sinaimg.cn/large/007S8ZIlgy1ghu6d9c19jj30vr0u0u0x.jpg)

Pode-se ver que nossa máquina é apenas uma transferência de estilo simples e rude, e o próprio mestre vai recriar na imitação.

Finalmente, adicione mais um ponto. Este algoritmo de transferência de estilo é lento, mas bastante simples. Para obter uma transferência rápida de estilo, considere: Primeiro, use o método apresentado aqui para corrigir uma imagem de referência de estilo, gere muitos exemplos de treinamento de "entrada-saída" para imagens de conteúdo diferentes e use esses "dados de entrada-saída" Treine uma rede neural convolucional simples para aprender este estilo particular de transformação (entrada -> saída). Após a conclusão, a transferência de um estilo específico para uma imagem é muito rápida e um passe para frente é concluído.